物性値を返すクラスの例です。大分迷いましたが、クラス概念を使う方が、呼び出し元のプログラムが非常に簡潔にかけるため、がんばって勉強してください。

例として、合板(plywood)とグラスウール(glasswool)とします。

各物性は、伝導率と容積比熱を返す関数を持つとします。

伝導率と容積比熱は、温度と水分を表すパラメータの2つを引数とするとします。水分を表すパラメータの採用候補としては、相対湿度=RHと含水率=psiとします。

伝導率を取得する関数はCond、容積比熱を取得する関数はCRowとします。

たまたま、合板はRH基準で、グラスウールは含水率基準で表したとします。

## 各モジュールに書くこと

### Plywoodモジュール

In [1]:
class Plywood():
    
    def __init__( self ):
        self.moistureIndex = 'RH' # 水分を表す指標としてRHを採用する場合は、'RH'を返すこととします。
        
    def Cond( self, temp, rh ):
        return 1.2   # 返り値はダミー。本来であれば、tempとRHの関数。
    
    def CRow( self, temp, rh ):
        return 500.0   # 返り値はダミー。本来であれば、tempとRHの関数。

#### 使い方

In [2]:
p = Plywood()

In [3]:
p.Cond( temp = 12.0, rh = 0.6 )

1.2

In [4]:
p.CRow( temp = 12.0, rh = 0.6 )

500.0

### グラスウールモジュール

In [5]:
class GlassWool():
    
    def __init__( self ):
        self.moistureIndex = 'psi' # 水分を表す指標としてpsiを採用する場合は、'psiを返すこととします。
    
    def Cond( self, temp, psi ):
        return 1.8   # 返り値はダミー。本来であれば、tempとpsiの関数。
    
    def CRow( self, temp, psi ):
        return 700.0   # 返り値はダミー。本来であれば、tempとpsiの関数。

#### 使い方

In [6]:
gw = GlassWool()

In [7]:
gw.Cond( temp = 15.0, psi = 100.0 )

1.8

In [8]:
gw.CRow( temp = 15.0, psi = 100.0 )

700.0

ここまで、実際にはクラス１つについて１つのモジュールとして記述します。

## 各モジュールを統括するモジュール

物性値を管理するプログラムの例です。

このモジュールは一度書いてしまえば、getMaterial関数に新たに物性値を追加するとき以外は、変更しません。

RHからpsiに変換する関数が下記のようにあったとします。

In [9]:
def RH_to_Psi( rh ):
    psi = rh # ダミー。本来であれば、何らかの関数により換算される。
    return psi

psiからRHに変換する関数が下記のようにあったとします。

In [10]:
def Psi_to_RH( psi ):
    rh = psi # ダミー。本来であれば、何らかの関数により換算される。
    return rh

水分を表すパラメータとしてrhかpsiかを判別するための関数です。material.moistureIndex を見に行ってそれが、'RH'なのか'psi'なのかで換算をかけるかかけないかを判断しています。

In [11]:
def RH_to_MoistureValue( material, rh ):
    if material.moistureIndex == 'RH':
        return rh
    elif material.moistureIndex == 'psi':
        return RH_to_Psi(rh)
    else:
        raise ValueError('各物性クラスのmoistureIndexには、「RH」か「psi」を指定しください。')

In [12]:
def Psi_to_MoistureValue( material, psi ):
    if material.moistureIndex == 'RH':
        return Psi_to_RH(psi)
    elif material.moistureIndex == 'psi':
        return psi
    else:
        raise ValueError('各物性クラスのmoistureIndexには、「RH」か「psi」を指定しください。')

tempとRHからCondを計算する関数。

In [13]:
def CondAsRH( material, temp, rh ):
    return material.Cond( temp, RH_to_MoistureValue( material, rh ) )

tempとpsiからCondを計算する関数

In [14]:
def CondAsPsi( material, temp, psi ):
    return material.Cond( temp, Psi_to_MoistureValue( material, psi ) )

tempとRHからCROWを計算する関数

In [15]:
def CRowAsRH( material, temp, rh ):
    return material.CRow( temp, RH_to_MoistureValue( material, rh ) )

tempとpsiからCROWを計算する関数

In [16]:
def CRowAsPsi( material, temp, psi ):
    return material.CRow( temp, Psi_to_MoistureValue( material, psi ) )

引数'name'に基づいて、該当するクラスを返します。

In [17]:
def getMaterial( name ):
    
    # 新しく物性モジュールを作成した場合は、ここに追加してください。
    mlist = {
        '合板' : Plywood,
        'グラスウール' : GlassWool
    }
    
    M = mlist[name]
    
    # 上記で定義した関数を動的にクラス'M'に追加します。
    M.CondAsRH  = CondAsRH
    M.CondAsPsi = CondAsPsi
    M.CRowAsRH  = CRowAsRH
    M.CRowAsPsi = CRowAsPsi
    
    return M()

使い方

getMaterial関数の引数に取得したい物性値名称を指定して、物性値を取得します。

In [18]:
m1 = getMaterial('合板')

例えば、Psi基準でCondを取得したかったら・・・

In [19]:
m1.CondAsPsi( temp = 15.0, psi = 25.0 )

1.2

RH基準でCondを取得したかったら・・・

In [20]:
m1.CondAsRH( temp = 15.0, rh = 0.6 )

1.2

CRowも同様に

In [21]:
m1.CRowAsPsi( temp = 15.0, psi = 25.0 )

500.0

In [22]:
m1.CRowAsRH( temp = 15.0, rh = 0.6 )

500.0

グラスウールの場合も同様に

In [23]:
m2 = getMaterial('グラスウール')

In [24]:
m2.CondAsPsi( temp = 15.0, psi = 25.0 )

1.8

In [25]:
m2.CondAsRH( temp = 15.0, rh = 0.6 )

1.8

In [26]:
m2.CRowAsPsi( temp = 15.0, psi = 25.0 )

700.0

In [27]:
m2.CRowAsRH( temp = 15.0, rh = 0.6 )

700.0

なお、クラスに関数をあとから追加するという概念がわかりにくければ、以下のように書いてしまってもかまいません。

In [28]:
m3 = getMaterial('合板')

In [29]:
CondAsPsi( material = m3, temp = 15.0, psi = 25.0 )

1.2

ただ、  
CondAsPsi( m3, 15.0, 25.0 )  
と書くのと  
m3.CondAsPsi( 15.0, 25.0 )  
と書くのでは、後者の方が、主語述語がはっきりしていて、慣れればすっきり見えるので、なるべく後者が望ましいです。

参考までに・・・・  
あとからクラスに関数を追加するという高度なことをやっていますが、通常のクラスの関数は以下のように書きます。

In [30]:
class Sample():
    
    def __init__( self, x ):
        self.x = x
    
    def SampleMethod( self, y ):
        return self.x + y

In [31]:
s = Sample( 3 )

In [32]:
s.SampleMethod( 2 )

5

クラスの中の関数の第1引数は自分自身ですので、  
s.SampleMethod( 2 )  と  SampleMethod( s, 2 )  は、実はまったく同じ意味を持ちます。